In [ ]:
# ==============================================================================
# CÉLULA 1: SETUP E INICIALIZAÇÃO DA ANÁLISE
# ==============================================================================
# Descrição: Importa bibliotecas, configura o ambiente e inicializa o
# AnalisadorBancario, que será a ferramenta para todas as coletas de dados.
# ------------------------------------------------------------------------------
import pandas as pd
from pathlib import Path
from bacen_analysis import AnalisadorBancario

print("--- 1. INICIANDO SETUP ---")

# Configurações de exibição do Pandas
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100)

# Inicializa o Analisador (carrega todos os dados em memória)
output_dir = Path('../../data/output').resolve()
analisador = AnalisadorBancario(diretorio_output=str(output_dir))

# Garante que a pasta de resultados exista
results_dir = Path('../../data/results').resolve()
results_dir.mkdir(parents=True, exist_ok=True)

print("--- SETUP CONCLUÍDO. ANALISADOR PRONTO. ---")

In [ ]:
# ==============================================================================
# CÉLULA 2: PAINEL DE CONTROLE DA ANÁLISE
# ==============================================================================
# Descrição: Este é o único lugar que você precisa alterar para configurar
# todo o relatório. Defina aqui os bancos, indicadores e o período desejado.
# ------------------------------------------------------------------------------
print("--- 2. CONFIGURANDO PARÂMETROS DA ANÁLISE ---")

# --- Período da Análise ---
DATA_INICIO_SERIES = 202501
DATA_FIM_SERIES = 202506
DATA_SNAPSHOT = 202506

# --- Geração Inteligente de Datas ---
data_inicio_dt = pd.to_datetime(f'{DATA_INICIO_SERIES}', format='%Y%m')
data_fim_dt = pd.to_datetime(f'{DATA_FIM_SERIES}', format='%Y%m') + pd.tseries.offsets.MonthEnd(0)

datas_mensais = pd.date_range(
    start=data_inicio_dt,
    end=data_fim_dt,
    freq='ME'
).strftime('%Y%m').astype(int).tolist()

datas_trimestrais = pd.date_range(
    start=data_inicio_dt,
    end=data_fim_dt,
    freq='QE'
).strftime('%Y%m').astype(int).tolist()

# --- Lista de Bancos ---
try:
    caminho_csv = Path('.') / 'Banks.csv'
    df_bancos_interesse = pd.read_csv(caminho_csv, dtype={'CNPJ_8': str}, sep=';', encoding='latin-1')
    LISTA_BANCOS = [b for b in df_bancos_interesse['CNPJ_8'].tolist() if b]
except FileNotFoundError:
    print(f"ERRO CRÍTICO: O arquivo '{caminho_csv.name}' não foi encontrado. A análise será interrompida.")
    LISTA_BANCOS = []

# --- CONTROLE GLOBAL DE TRATAMENTO DE DADOS ---
MODO_TRATAMENTO_DADOS = 'preencher_com_zero'


# --- GRUPO 1: INDICADORES PARA SÉRIES TEMPORAIS (HISTÓRICO COMPLETO) ---
INDICADORES_SERIES_TEMPORAIS = {
    'Ativo Total': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'RWA': {'fonte': 'IFDATA', 'conta': 'Patrimônio de Referência para Comparação com o RWA (e)', 'escopo_ifdata': 'prudencial'},
    'Índice de Basileia': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Lucro Líquido': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Patrimônio Líquido': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Valor Contábil Bruto (e1)': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Valor Contábil Bruto (f1)': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Valor Contábil Bruto (g1)': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Valor Contábil Bruto (h1)': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Perda Esperada (e2)': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Perda Esperada (f2)': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Perda Esperada (g2)': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'},
    'Perda Esperada (h2)': {'fonte': 'IFDATA', 'escopo_ifdata': 'prudencial'}
}

# --- GRUPO 2: INDICADORES PARA SNAPSHOT (APENAS DATA MAIS RECENTE) ---
INDICADORES_SNAPSHOT = {
        'Carteira de Clientes C1': {'fonte': 'IFDATA', 'conta': 'C1'},
        'Carteira de Clientes C2': {'fonte': 'IFDATA', 'conta': 'C2'},
        'Carteira de Clientes C3': {'fonte': 'IFDATA', 'conta': 'C3'},
        'Carteira de Clientes C4': {'fonte': 'IFDATA', 'conta': 'C4'},
        'Carteira de Clientes C5': {'fonte': 'IFDATA', 'conta': 'C5'},
        'Depósitos a Prazo': {'fonte': 'IFDATA', 'conta': 'Depósitos a Prazo (a4)', 'escopo_ifdata': 'prudencial'},
        'Letra de Crédito Imobiliário': {'fonte': 'IFDATA', 'conta': 'Letras de Crédito Imobiliário (c1)', 'escopo_ifdata': 'prudencial'},
        'Letra de Crédito Agronegócio': {'fonte': 'IFDATA', 'conta': 'Letras de Crédito do Agronegócio (c2)', 'escopo_ifdata': 'prudencial'},
        'Letras Financeiras': {'fonte': 'IFDATA', 'conta': 'Letras Financeiras (c3)', 'escopo_ifdata': 'prudencial'}
        }

# --- ATRIBUTOS CADASTRAIS (DADOS ESTÁTICOS) ---
ATRIBUTOS_CADASTRAIS = ['TCB_IFD_CAD']

print("--- CONFIGURAÇÃO CARREGADA. ---")

In [ ]:
# ==============================================================================
# CÉLULA 3: COLETA OTIMIZADA E EXPORTAÇÃO DIRETA (BATCH PROCESSING)
# ==============================================================================
# Descrição: Pipeline otimizado usando batch processing para máxima performance.
# Reduz drasticamente o número de chamadas de função através de processamento em lote.
# ------------------------------------------------------------------------------

if LISTA_BANCOS:
    todos_indicadores = {**INDICADORES_SERIES_TEMPORAIS, **INDICADORES_SNAPSHOT}
    
    # Traduz o modo de tratamento global para parâmetros de função
    parametros_tratamento = {}
    if MODO_TRATAMENTO_DADOS == 'remover_nulos':
        parametros_tratamento = {'drop_na': True}
    elif MODO_TRATAMENTO_DADOS == 'preencher_com_zero':
        parametros_tratamento = {'fill_value': 0, 'drop_na': False}
    elif MODO_TRATAMENTO_DADOS == 'manter_nulos':
        parametros_tratamento = {'drop_na': False}
    elif MODO_TRATAMENTO_DADOS == 'remover_zeros_e_nulos':
        parametros_tratamento = {'replace_zeros_with_nan': True, 'drop_na': True}
    
    # Construção da lista de requisições para batch processing
    requisicoes = []
    
    for banco in LISTA_BANCOS:
        for nome_indicador, info in todos_indicadores.items():
            # Determina a lista de datas correta para cada indicador
            if nome_indicador in INDICADORES_SERIES_TEMPORAIS:
                datas_para_buscar = (
                    datas_trimestrais 
                    if info['fonte'].upper() == 'IFDATA' 
                    else datas_mensais
                )
            else:
                datas_para_buscar = [DATA_SNAPSHOT]
            
            # Constrói requisição
            req = {
                'identificador': banco,
                'conta': info.get('conta', nome_indicador),
                'fonte': info['fonte'],
                'datas': datas_para_buscar,
                'nome_indicador': nome_indicador
            }
            
            # Adiciona parâmetros opcionais apenas se definidos
            if 'documento_cosif' in info:
                req['documento_cosif'] = info['documento_cosif']
            if 'escopo_ifdata' in info:
                req['escopo_ifdata'] = info['escopo_ifdata']
            
            requisicoes.append(req)
    
    print("Coletando dados...")
    
    # CHAMADA ÚNICA EM LOTE - Substitui centenas de chamadas individuais
    df_long_base = analisador.get_series_temporais_lote(
        requisicoes,
        **parametros_tratamento
    )
    
    if df_long_base.empty:
        print("Nenhum dado foi encontrado para os parâmetros definidos.")
    else:
        # Renomeia a coluna 'Conta' para 'Indicador' (já está com nome_indicador)
        df_long_base.rename(columns={'Conta': 'Indicador'}, inplace=True)
        
        # Busca atributos cadastrais
        df_atributos = analisador.get_atributos_cadastro(
            identificador=LISTA_BANCOS, 
            atributos=ATRIBUTOS_CADASTRAIS
        )
        
        # Merge com atributos cadastrais
        df_para_bi = pd.merge(
            df_long_base, 
            df_atributos, 
            on=['CNPJ_8', 'Nome_Entidade'], 
            how='left'
        )
        
        # Organiza colunas finais
        colunas_finais = ['DATA', 'Nome_Entidade', 'CNPJ_8', 'Indicador', 'Valor'] + [
            col for col in ATRIBUTOS_CADASTRAIS if col in df_para_bi.columns
        ]
        df_para_bi = df_para_bi[colunas_finais]
        
        # Exporta para Excel
        nome_arquivo_saida = f'relatorio_powerbi_202506.xlsx'
        caminho_saida = results_dir / nome_arquivo_saida
        df_para_bi.to_excel(caminho_saida, index=False, engine='openpyxl')
        
        print("Relatório gerado com sucesso!")
        print(f"Arquivo: {caminho_saida.resolve()}")
        print(f"Linhas exportadas: {len(df_para_bi):,}")
else:
    print("A lista de bancos está vazia. Nenhuma análise foi executada.")